In [10]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import os
from pathlib import Path
import numpy as np

In [11]:
path = Path(input("请输入路径："))

In [12]:
os.chdir(path)
print(f"当前路径为:{os.getcwd()}")

当前路径为:/Users/wangshuaibo/Documents/北京悦动/脑卒中项目/阿帅笔记/测试数据及分析结果/20230421医院鞋垫数据检验


In [13]:
for file in os.listdir():
    if ".csv" in file:
        print(file)


hwshuai_丁冠宏-走40步路-鞋垫4-1-604F-V4.16-014_202304211216193530.csv
hwshuai_丁冠宏-走40步路-鞋垫4-1-604F-V4.16-013_202304211209071330.csv


In [14]:
file = "hwshuai_丁冠宏-走40步路-鞋垫4-1-604F-V4.16-014_202304211216193530.csv"

In [18]:
# 走40步路
file = "/Users/wangshuaibo/Documents/北京悦动/脑卒中项目/阿帅笔记/测试数据及分析结果/20230420鞋垫测试/hwshuai_丁冠宏-走6步路-鞋垫2-1-41AB-V4.16-006_202304201530532020.csv"

In [16]:
# 跑步机走 10 分钟
file = "/Users/wangshuaibo/Documents/北京悦动/脑卒中项目/阿帅笔记/测试数据及分析结果/20230421跑步机走路测试/Done/hwshuai_丁冠宏-跑步机走路10分钟-鞋垫2-1-41AB-V4.01-002_202304211610535380.csv"

In [19]:
df = pd.read_csv(file)
L_data = df[df["part"]=="L"]
R_data = df[df["part"]=="R"]
R_data.columns

Index(['part', 'timestamp', 'gx', 'gy', 'gz', 'ax', 'ay', 'az', 'qw', 'qx',
       'qy', 'qz', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10',
       'p11', 'p12', 'p1-c', 'p2-c', 'p3-c', 'p4-c', 'p5-c', 'p6-c', 'p7-c',
       'p8-c', 'p9-c', 'p10-c', 'p11-c', 'p12-c'],
      dtype='object')

In [28]:
# 传感器列表
sensors = ['p1-c', 'p2-c', 'p3-c', 'p4-c', 'p5-c', 'p6-c', 'p7-c',
       'p8-c', 'p9-c', 'p10-c', 'p11-c', 'p12-c']


In [29]:
def SensorsCalibre(part_data,part):
    sensors_calibre = pd.DataFrame({})

    for sensor_name in sensors:
        sensor = part_data[sensor_name]

        # 查找峰值
        # 以快跑 16km/h ，对应至少间隔 22 个数
        peaks,_ = find_peaks(-sensor,-sensor.max(),width=20)
        L_shape = sensor.shape
        peakls = peaks.tolist()

        # 查找峰值并绘制图片
        #sensor.plot()
        #plt.plot(peaks*2,sensor.iloc[peakls])

        # 使用峰值校准数据
        # 所有数据减去前一个峰值
        # 第一个峰值之前的数据减去第一个峰值
        range_start = peakls[0]
        df_calibre = pd.DataFrame({})
        # 选取区间内数值
        snip_data= sensor.iloc[0:range_start]
        # 进行校准
        calibre_sensor_snip = snip_data.apply(lambda x: x-sensor.iloc[range_start])
        df_calibre = pd.concat([df_calibre.reset_index(drop=True), 
                                        calibre_sensor_snip.reset_index(drop=True)],axis=0)   
        for i in range(len(peakls)):

            # 选取两个波谷之间的区间
            range_start = peakls[i]
            if i < len(peakls)-1:
                range_end = peakls[i+1]
            else:
                range_end = L_shape[0]
            

            # 选取区间内数值
            snip_data= sensor.iloc[range_start:range_end]
            # 进行校准
            calibre_sensor_snip = snip_data.apply(lambda x: x-sensor.iloc[range_start])
            

            # 将当前生成的 Series 与 df_calibre 拼接
            df_calibre = pd.concat([df_calibre.reset_index(drop=True), 
                                        calibre_sensor_snip.reset_index(drop=True)],axis=0)
            
            # 重置一下索引，使得索引是唯一的
            df_calibre.reset_index(drop=True, inplace=True)
            
        df_calibre.columns = [sensor_name]
        sensors_calibre = pd.concat([sensors_calibre.reset_index(drop=True),
                            df_calibre.reset_index(drop=True)],axis=1)

        # 绘图
        # sensor.plot()
        # df_calibre.plot()
        
    sensors_calibre.insert(0,"part", part)

    return sensors_calibre

In [30]:
L_SensorsCalibre = SensorsCalibre(L_data,"L")
R_SensorsCalibre = SensorsCalibre(R_data,"R")

In [32]:
SensorsCalibre_df = pd.concat([L_SensorsCalibre.reset_index(drop=True),
                               R_SensorsCalibre.reset_index(drop=True)])

In [38]:
SensorsCalibre_df.to_csv("test.csv")

In [35]:
os.getcwd()

'/Users/wangshuaibo/Documents/北京悦动/脑卒中项目/阿帅笔记/测试数据及分析结果/20230421医院鞋垫数据检验'

In [ ]:
# 新建动态校准后文件夹
DynamicCalibreDir = "DynamicCalibre"
if not os.path.exists(DynamicCalibreDir):
    os.mkdir(DynamicCalibreDir)

pd.to_csv()